![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

### [Docker, Spark, and Iceberg: The Fastest Way to Try Iceberg!](https://tabular.io/blog/docker-spark-and-iceberg/)

In [7]:
!pip install pyspark-opendic==0.3.18

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

def read_secret(secret_name):
    """ Get `secret_name` from docker-compose secret store"""
    secret_path = f"/run/secrets/{secret_name}"
    try:
        with open(secret_path, "r") as f:
            return f.read().strip()  # Remove any trailing newline
    except FileNotFoundError:
        print(f"Secret {secret_name} not found.")
        return None


## DEFINE SENSITIVE VARIABLES
POLARIS_CATALOG_NAME = 'AZURE_CATALOG'
ENGINEER_CLIENT_ID = read_secret("engineer_client_id")
ENGINEER_CLIENT_SECRET =  read_secret("engineer_client_secret")
ADLS_IO="org.apache.iceberg.azure.adlsv2.ADLSFileIO"
FILE_IO="org.apache.iceberg.io.ResolvingFileIO"

def create_session(client_id, client_secret, scope, fileio_impl):
    spark = (SparkSession.builder
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0,software.amazon.awssdk:bundle:2.28.17,software.amazon.awssdk:url-connection-client:2.28.17")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
        .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.polaris.type", "rest")
        .config("spark.sql.catalog.polaris.warehouse", POLARIS_CATALOG_NAME)
        .config("spark.sql.catalog.polaris.uri", 'https://opendict.duckdns.org/api/catalog')
        .config("spark.sql.catalog.polaris.credential", f"{client_id}:{client_secret}")
        .config("spark.sql.catalog.polaris.scope", 'PRINCIPAL_ROLE:ALL')
        .config("spark.sql.defaultCatalog", "polaris")
        .config("spark.sql.catalogImplementation", "in-memory")
        .config("spark.driver.extraJavaOptions", "-Divy.cache.dir=/tmp -Divy.home=/tmp")
        .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
        .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')
        .config("spark.sql.catalog.polaris.io-impl", fileio_impl)
        .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()
        
    print("Spark Running")
    return spark


## Start Spark Session
spark = create_session(client_id=ENGINEER_CLIENT_ID, client_secret=ENGINEER_CLIENT_SECRET, scope='PRINCIPAL_ROLE:ALL',fileio_impl=ADLS_IO )
spark

Spark Running


25/06/08 17:56:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Setting up pyspark-opendic

In [10]:
from pyspark_opendic.catalog import OpenDicCatalog

# Init polarisx catalog
POLARIS_URI= "https://opendict.duckdns.org/api"
catalog = OpenDicCatalog(spark, POLARIS_URI)
print("Catalog initialized")

Catalog initialized


In [11]:
catalog.sql("Show namespaces").show()

+---------+
|namespace|
+---------+
|      nyc|
|   SYSTEM|
+---------+



In [23]:
%%sql
SELECT current_namespace()

AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `current_namespace` on search path [`system`.`builtin`, `system`.`session`, `polaris`].; line 1 pos 7

### Define the schema for a andfunc object

In [12]:
catalog.sql(
    """
    DEFINE OPEN function_v2
    props {
        "args": "MAP",
        "language": "STRING",
        "def": "string",
        "comment": "string",
        "packages": "list",
        "runtime": "string",
        "client_version": "int",
        "signature": "STRING",
        "return_type": "STRING"
    }
    """
)

,udoType,properties,createdTimestamp,lastUpdatedTimestamp,version
0,function_v2,"{'return_type': 'STRING', 'created_time': 'STRING', 'entity_version': 'STRING', 'uname': 'STRING', 'def': 'STRING', 'signature': 'STRING', 'runtime': 'STRING', 'language': 'STRING', 'packages': 'STRING', 'args': 'STRING', 'last_updated_time': 'STRING', 'comment': 'STRING', 'client_version': 'STRING'}",1970-01-01T00:00Z,1970-01-01T00:00Z,None


### Create a new andfunc

In [14]:
catalog.sql(
 """
 CREATE OPEN function_v2 foo
    props {
            "args": {
                "arg1": "int", 
                "arg2": "int"
                },
            "language": "python",
            "def": "def foo(arg1, arg2):\\n\\n        return arg1 + arg2",
            "packages" : ["numpy", "pandas"],
            "comment": "test fun",
            "runtime": "3.12",
            "client_version": 1,
            "return_type": "int",
            "signature": "foo(arg1 str, arg2 int)"
        }
    """
)

```json
{
    "error": "HTTP Error",
    "details": "409 Client Error: Conflict for url: https://opendict.duckdns.org/api/opendic/v1/objects/function_v2",
    "Catalog Response": null
}
```

### Create Mapping to snowflake

In [ ]:
catalog.sql(
    """
    ADD OPEN MAPPING function_v2 PLATFORM snowflake
    SYNTAX {
        "CREATE OR ALTER <type> <name>(<args>)
            RETURNS <return_type>
            LANGUAGE <language>
            PACKAGES = (<packages>)
            runtime_version = <runtime>
            HANDLER = '<name>'
            AS 
        $$
        <def>
        $$;",
    }
    PROPS {
        "args": {
                "propType": "map",
                "format": "<key> <value>",
                "delimiter": ", "
            },
        "packages": {"propType": "list", "format": "'<item>'", "delimiter": ", "}
    }
    """
)

### List objects

In [21]:
catalog.sql(
    """
    SHOW OPEN TYPES
    """
)

""


In [32]:
catalog.sql(
    """
    SHOW OPEN function_v2
    """
)

,type,name,props,createdTimestamp,lastUpdatedTimestamp,entityVersion
0,function_v2,foo,"{'args': {'arg1': 'int', 'arg2': 'int'}, 'return_type': 'int', 'def': 'def foo(arg1, arg2): return arg1 + arg2', 'signature': 'foo(arg1 str, arg2 int)', 'runtime': '3.12', 'language': 'python', 'comment': 'test fun', 'packages': ['numpy', 'pandas'], 'client_version': 1}",2025-06-07T14:52:45.212726551Z,2025-06-07T14:52:45.212734967Z,1


In [ ]:
# Show mapping for <object> to <platform>. Example: [Platform_mapping(function_v2 -> snowflake)]
catalog.sql(
    """
    SHOW OPEN MAPPING function_v2 PLATFORM snowflake
    """
)

In [ ]:
# Show all mappings from <object>. Example: [snowflake,spark]
catalog.sql(
    """
    SHOW OPEN PLATFORMS FOR function_v2
    """
)

In [ ]:
catalog.sql(
    """
    SHOW OPEN PLATFORMS
    """
)

In [ ]:
catalog.sql(
    """
    SHOW OPEN MAPPINGS FOR snowflake
    """
)

In [ ]:
catalog.sql(
    """
    SYNC OPEN function_v2 for snowflake
    """
)

In [ ]:
catalog.sql(
    """
    SYNC OPEN OBJECTS for snowflake
    """
)

### Drop andfunc

In [ ]:
catalog.sql(
    """
    DROP OPEN function_v2
    """
)


In [ ]:
catalog.sql(
    """
    DROP OPEN MAPPINGS for snowflake
    """
)


### Visualize opendic tables

In [15]:
%%sql
show tables in SYSTEM

namespace,tableName,isTemporary
SYSTEM,function_v2,False


In [16]:
%%sql
select uname as name, args, return_type, signature, runtime, language, comment, packages, def   from SYSTEM.function_v2

name,args,return_type,signature,runtime,language,comment,packages,def
foo,"{'arg2': 'int', 'arg1': 'int'}",int,"foo(arg1 str, arg2 int)",3.12,python,test fun,"['numpy', 'pandas']","def foo(arg1, arg2): return arg1 + arg2"


In [15]:
%%sql
USE SYSTEM

++
||
++
++

In [19]:
%%sql
select * from SYSTEM.function_v2

uname,args,return_type,def,signature,runtime,language,comment,packages,client_version,created_time,last_updated_time,entity_version
foo,"{'arg2': 'int', 'arg1': 'int'}",int,"def foo(arg1, arg2): return arg1 + arg2","foo(arg1 str, arg2 int)",3.12,python,test fun,"['numpy', 'pandas']",1,2025-06-08T17:58:27.161420711Z,2025-06-08T17:58:27.161423271Z,1


## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [2]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

25/06/08 17:56:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/06/08 17:56:53 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to https://opendict.duckdns.org/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


++
||
++
++

In [3]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

++
||
++
++

In [4]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis")

25/06/08 17:56:58 WARN ADLSFileIO: Failed to delete path: abfss://polarisbucket@polarisstorageacc.blob.core.windows.net/warehouse/nyc/taxis/data/00000-1-f18c8544-c33f-4f11-b870-626609e33420-0-00001.parquet
com.azure.storage.file.datalake.models.DataLakeStorageException: Status code 404, "{"error":{"code":"PathNotFound","message":"The specified path does not exist.\nRequestId:1efb88b8-e01f-0000-6d9e-d8e251000000\nTime:2025-06-08T17:56:58.4021615Z"}}"
	at com.azure.storage.file.datalake.implementation.util.ModelHelper.mapToDataLakeStorageException(ModelHelper.java:210)
	at com.azure.storage.file.datalake.implementation.PathsImpl.deleteWithResponse(PathsImpl.java:5813)
	at com.azure.storage.file.datalake.DataLakePathClient.lambda$deleteWithResponse$1(DataLakePathClient.java:643)
	at com.azure.storage.common.implementation.StorageImplUtils.sendRequest(StorageImplUtils.java:494)
	at com.azure.storage.file.datalake.DataLakePathClient.deleteWithResponse(DataLakePathClient.java:651)
	at com.az

In [5]:
%%sql

DESCRIBE EXTENDED nyc.taxis

col_name,data_type,comment
VendorID,bigint,None
tpep_pickup_datetime,timestamp_ntz,None
tpep_dropoff_datetime,timestamp_ntz,None
passenger_count,double,None
trip_distance,double,None
RatecodeID,double,None
store_and_fwd_flag,string,None
PULocationID,bigint,None
DOLocationID,bigint,None
payment_type,bigint,None


In [6]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

cnt
2171187


In [20]:
%%sql

SELECT *
FROM nyc.taxis limit 10

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2021-04-01 00:00:18,2021-04-01 00:21:54,1.0,8.4,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,0.3,35.15,2.5,0.0
1,2021-04-01 00:42:37,2021-04-01 00:46:23,1.0,0.9,1.0,N,75,236,2,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,0.0
1,2021-04-01 00:57:56,2021-04-01 01:08:22,1.0,3.4,1.0,N,236,168,2,11.5,3.0,0.5,0.0,0.0,0.3,15.3,2.5,0.0
1,2021-04-01 00:01:58,2021-04-01 00:54:27,1.0,0.0,1.0,N,47,61,1,44.2,0.0,0.5,0.0,0.0,0.3,45.0,0.0,0.0
2,2021-04-01 00:24:55,2021-04-01 00:34:33,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,0.3,13.39,0.0,0.0
2,2021-04-01 00:19:16,2021-04-01 00:21:46,1.0,0.77,1.0,N,142,238,1,4.5,0.5,0.5,1.24,0.0,0.3,9.54,2.5,0.0
2,2021-04-01 00:25:11,2021-04-01 00:31:53,1.0,3.65,1.0,N,238,244,1,11.5,0.5,0.5,2.56,0.0,0.3,15.36,0.0,0.0
1,2021-04-01 00:27:53,2021-04-01 00:47:03,0.0,8.9,1.0,N,138,239,1,26.5,3.0,0.5,7.25,6.12,0.3,43.67,2.5,0.0
2,2021-04-01 00:24:24,2021-04-01 00:37:50,1.0,2.98,1.0,N,151,244,2,12.0,0.5,0.5,0.0,0.0,0.3,13.3,0.0,0.0
1,2021-04-01 00:19:18,2021-04-01 00:41:25,1.0,8.9,1.0,N,132,196,2,28.0,0.5,0.5,0.0,0.0,0.3,29.3,0.0,0.0


## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [ ]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN fare_amount TO fare

In [ ]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN trip_distance TO distance

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance TYPE double;

In [ ]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance AFTER fare;

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [ ]:
%%sql

UPDATE nyc.taxis
SET fare_per_distance_unit = fare/distance

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

## Expressive SQL for Row Level Changes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 or a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit is null

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

## Partitioning

A table’s partitioning can be updated in place and applied only to newly written data. Query plans are then split, using the old partition scheme for data written before the partition scheme was changed, and using the new partition scheme for data written after. People querying the table don’t even have to be aware of this split. Simple predicates in WHERE clauses are automatically converted to partition filters that prune out files with no matches. This is what’s referred to in Iceberg as *Hidden Partitioning*.

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD PARTITION FIELD VendorID

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [ ]:
%%sql

SELECT snapshot_id, manifest_list
FROM nyc.taxis.snapshots

The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [ ]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM nyc.taxis.files

## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [ ]:
%%sql --var df

SELECT *
FROM nyc.taxis.history

In [ ]:
original_snapshot = df.head().snapshot_id
spark.sql(f"CALL system.rollback_to_snapshot('nyc.taxis', {original_snapshot})")
original_snapshot

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis